In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd

# Setting up the Chrome WebDriver
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)
url = "https://www.inmotionhosting.com"
driver.get(url)

# Extract HTML content
html_content = driver.page_source

# Use BeautifulSoup to parse HTML
soup = BeautifulSoup(html_content, "html.parser")
data = {'Plan name': [],
        'Summ':[],
        'Price': [],
        'Periods_of_payment': [],
        'Currency': []}
plans_table = soup.find('div',class_='imh-rostrum-container')
plans_items = soup.find_all('div',class_='imh-rostrum-card')
try:
    for plan_item in plans_items:
        if plan_item.find('div',class_='imh-rostrum-sub-title'):
            summury = plan_item.find('div',class_='imh-rostrum-sub-title').text.strip()
        else:
            summury = 'None'
        print(type(summury))
        plan_name = plan_item.find('h3').text.strip()
        price_details = plan_item.find('div',class_='imh-rostrum-starting-at-price-discounted').text.split('/')
        currency = price_details[0][0]  # e.g., '$'
        price = price_details[0][1:]  # e.g., 6.99
        period = price_details[1]
        data['Plan name'].append(plan_name)
        data['Summ'].append(summury)
        data['Currency'].append(currency)
        data['Price'].append(price)
        data['Periods_of_payment'].append(period)
        # Creating a DataFrame
        df_hosting = pd.DataFrame(data)

except (NoSuchElementException, TimeoutException) as e:
    print("Error: ", e)
finally:
    # Closing the driver
    driver.quit()


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [2]:
df_hosting = df_hosting[df_hosting['Summ'] != 'None']

In [3]:
df_hosting

,Plan name,Summ,Price,Periods_of_payment,Currency
0,Shared Hosting,Cost effective and fast for small businesses,2.49,mo,$
1,VPS Hosting,For high traffic sites and larger businesses,14.99,mo,$
2,Dedicated Hosting,For the largest sites and custom deployments,69.99,mo,$
3,cPanel WordPress,Expertly tuned hosting with cPanel,3.99,mo,$
4,Platform i Cloud,Next generation managed WordPress,9.00,mo,$


In [4]:
df_hosting.to_csv('hosting_data.csv', index=False)